In [ ]:
# import necessary libraries and modules
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import os
import cv2
import imghdr
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, SparseCategoricalAccuracy
from tensorflow.keras.models import load_model

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
# define the data directory path
data_dir = './input/formula-one-cars-images'

In [ ]:
# build image dataset pipeline
data = tf.keras.utils.image_dataset_from_directory('./input/formula-one-cars-images/train/')

In [ ]:
# convert dataset to numpy iterator for easy access of data as a pipeline
data_iterator = data.as_numpy_iterator()

In [ ]:
# accessing data through extractaction of a single batch from the dataset
batch = data_iterator.next()

In [ ]:
# get the class names from the dataset
class_names = data.class_names

In [ ]:
# create a mapping from label index to class names
label_to_class = {i: class_name for i, class_name in enumerate(class_names)}
label_to_class

In [ ]:
# convert batch labels to class names
batch_class_names = [label_to_class[label] for label in label_to_class]
batch_class_names

In [ ]:
# scale the image data to range [0, 1]
scaled_data = data.map(lambda x,y: (x/255, y))

In [ ]:
# partition train
train_size = int(len(scaled_data)*0.7)

In [ ]:
# partition test
test_size = int(len(scaled_data)*0.1)+1

In [ ]:
# partition validation
val_size = int(len(scaled_data)*0.2)+1

In [ ]:
# split data
train = scaled_data.take(train_size)
val = scaled_data.skip(train_size).take(val_size)
test = scaled_data.skip(train_size+val_size).take(test_size)

In [ ]:
# initialize Sequential model
model = Sequential()

In [ ]:
# add convolutional and pooling layers
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())


# add dense layers for classification
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
# compile model
model.compile('adam', loss=tf.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
# get summary of model
model.summary()

In [ ]:
summary = str(model.to_json())
import json
with open('./models/model_summary.json', 'w') as f:
    json.dump(summary, f)

In [ ]:
# set the log directory to view history
logdir ='./logs'

In [ ]:
# set history
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
# show start time
import datetime
now = datetime.datetime.now()
print(now)

In [ ]:
# train model
model_training = model.fit(train, epochs=30, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
# show end time
now_end = datetime.datetime.now()
print(now_end)

In [ ]:
# visualise loss
fig = plt.figure()
plt.plot(model_training.history['loss'], color='teal', label='loss')
plt.plot(model_training.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc='upper left')
plt.savefig('./models/loss.png')

In [ ]:
# visualise accuracy
fig2 = plt.figure()
plt.plot(model_training.history['accuracy'], color='teal', label='accuracy')
plt.plot(model_training.history['val_accuracy'], color='orange', label='val_accuracy')
fig2.suptitle('Accuracy', fontsize=20)
plt.legend(loc='upper left')
plt.savefig('./models/accuracy.png')

In [ ]:
# set accuracy
accuracy = SparseCategoricalAccuracy()

In [ ]:
# get accuracy of model
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    accuracy.update_state(y, yhat)

In [ ]:
print('Accuracy:', accuracy.result().numpy())

In [ ]:
# test model on unforseen image - McLaren F1 Racecar
img = cv2.imread('./input/lando-norris-mclaren-mcl35m-1.png')
plt.imshow(img)
plt.show()

In [ ]:
# convert rgb
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
# plot fixed rgb image
plt.imshow(img_rgb)
plt.title("Original Image")
plt.show()

In [ ]:
# normalise for model
input_shape = (256, 256)
resized_img = cv2.resize(img_rgb, input_shape)
resized_img_normalized = resized_img / 255.0
input_data = np.expand_dims(resized_img_normalized, axis=0)

In [ ]:
# predict
yhat = model.predict(input_data)
yhat

In [ ]:
# map out the racecars
sorted_indices = np.argsort(yhat[0])[::-1]

In [ ]:
# print predictions
prefixes = ['st', 'nd', 'rd', 'th', 'th', 'th', 'th', 'th', 'th', 'th']
for i, index in enumerate(sorted_indices):
    prob = yhat[0][index]
    class_name = label_to_class[index]
    print(f'{i+1}{prefixes[i]} Prediction: {class_name} with {prob*100:.2f}% confidence.')

In [ ]:
# save model
model.save(os.path.join('./models','f1-racecar-image-classifier.h5'))

In [ ]:
model = load_model('./models/f1-racecar-image-classifier.h5')

In [ ]:
import mlflow
mlflow.set_tracking_uri(uri="http://models.f1-platform.zawalich.pl")

In [ ]:
from datetime import datetime
now = datetime.strptime("2024-03-27 18:04:14", "%Y-%m-%d %H:%M:%S")
now_end = datetime.strptime("2024-03-27 19:30:38", "%Y-%m-%d %H:%M:%S")
training_time = now_end - now
training_time.seconds


In [ ]:
accuracy = 0.98286605

In [ ]:
!pip freeze > environment.yaml

In [ ]:
import os
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "s3://minio.f1-platform.zawalich.pl:9000/"
print(os.environ["MLFLOW_S3_ENDPOINT_URL"])

In [ ]:
import getpass
os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass()

In [ ]:
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass()

In [ ]:
import time
# Define the model hyperparameters
params = {
    "classes": label_to_class,
    "params_size": "14.11 MB",
    "training_time_readable": time.strftime('%H:%M:%S', time.gmtime(training_time.seconds)),
    "training_time_start": print(now),
    "training_time_end": print(now_end)
}
# Create a new MLflow Experiment
mlflow.set_experiment("Formula One Image Classification Model")

# Start an MLflow run
with mlflow.start_run():
    
    mlflow.set_tag("mlflow.runName", "First initial run")

    # Log model
    mlflow.keras.log_model(model, artifact_path="formula1-model", registered_model_name="Formula One Image Classification Model")

    # Log the hyperparameters
    mlflow.log_params(params)

    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("epochs", 30)
    mlflow.log_metric("params", 3698938)
    mlflow.log_metric("training_time_seconds", training_time.seconds)
    mlflow.log_metric("layers", 9)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("First Training", "Training on Google Colaboratory, on CPU")

    # Log files
    mlflow.log_artifact('./environment.yaml', 'formula1-model')

    mlflow.log_artifact('./models/f1-racecar-image-classifier.h5', 'models')
    mlflow.log_artifact('./models/model_summary.json', 'models')
    mlflow.log_artifact('./models/loss.png', 'models')
    mlflow.log_artifact('./models/accuracy.png', 'models')   
    
    mlflow.log_artifact('./input/lando-norris-mclaren-mcl35m-1.png', 'input')
    mlflow.log_artifact('./input/formula-one-cars-images/train/alfa_romeo/20200427191026_1.png', 'input/formula-one-cars-images/train/alfa_romeo')
    mlflow.log_artifact('./input/formula-one-cars-images/train/bwt/20200427185603_1.png', 'input/formula-one-cars-images/train/bwt')
    mlflow.log_artifact('./input/formula-one-cars-images/train/ferrari/20200422160951_1.png', 'input/formula-one-cars-images/train/ferrari')
    mlflow.log_artifact('./input/formula-one-cars-images/train/haas/20200427181012_1.png', 'input/formula-one-cars-images/train/haas')
    mlflow.log_artifact('./input/formula-one-cars-images/train/mclaren/20200427183830_1.png', 'input/formula-one-cars-images/train/mclaren')
    mlflow.log_artifact('./input/formula-one-cars-images/train/mercedes/20200427164302_1.png', 'input/formula-one-cars-images/train/mercedes')
    mlflow.log_artifact('./input/formula-one-cars-images/train/redbull/20200427172058_1.png', 'input/formula-one-cars-images/train/redbull')
    mlflow.log_artifact('./input/formula-one-cars-images/train/renault/20200427174555_1.png', 'input/formula-one-cars-images/train/renault')
    mlflow.log_artifact('./input/formula-one-cars-images/train/toro_rosso/20200427202817_1.png', 'input/formula-one-cars-images/train/toro_rosso')
    mlflow.log_artifact('./input/formula-one-cars-images/train/williams/20200427204250_3.png', 'input/formula-one-cars-images/train/williams')

    mlflow.end_run()

In [ ]:
mlflow.end_run()